In [1]:
import pandas as pd
import re
import os
import sys
import pickle as pkl
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import json

# df_annotations = pd.read_csv("all_annotations.csv")


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


FileNotFoundError: ignored

In [4]:
df_annotations = pd.read_csv("/content/drive/MyDrive/data/all_annotations.csv")

In [5]:
!python --version

Python 3.7.15


# Get data

In [21]:
with open("/content/drive/MyDrive/data/protocols.pkl.gz", "rb") as f:
    file_to_pages = pkl.load(f)

In [22]:
import spacy
nlp = spacy.blank("en")

In [ ]:
texts = []

for i in range(len(df_annotations)):
    file_name = df_annotations.file.iloc[i]
    
    pages= file_to_pages[file_name]
    
    text = ""
    for page in pages:
        doc = nlp(page)
        
        is_include = [False] * len(doc)
        for tok in doc:
            if tok.text.lower() in {"phase", "arm", "arms", "armed", "cohort", "cohorts", "group", "groups",
                                   "sample",  "enroll", "enrol", "enrols", "enrolled", "enrolling", "enrolment",
                                    "recruit", "recruiting", "recruited", "recruits", "recruitment",
                                    "target", "accrual", "accruing", "accrue", "accrued",
                                    "n",
                                    # "overall", "total", 
                                    # "participants", "subjects", "people", "persons", "healthy", "infected",
                                    # "pairs", "individuals", "women", "men", "patients", "males", "females", "male", "female",
                                    # "select", "selection", "approximately", "achieve",
                                    # "cases"
                                   }:
                for token_index in range(tok.i - 20, tok.i + 20):
                    if token_index >= 0 and token_index < len(doc):
                        is_include[token_index] = True
        
        for token_index in range(len(doc)):
            if is_include[token_index]:
                text += doc[token_index].text + doc[token_index].whitespace_
                
                
    if text == "":
        print ("nothing found", file_name, df_annotations.phase.iloc[i])
        text = " ".join(pages)
        if len(text) > 100000:
            text = text[:100000]
    else:
        print ("found", file_name, df_annotations.phase.iloc[i], len(text))

    texts.append(text)
df_annotations["text"] = texts

Streaming output truncated to the last 5000 lines.
found 40_NCT02385240_Prot_001.pdf Phase 3 4114
found 40_NCT02407340_Prot_000.pdf Phase 1 7875
found 40_NCT02503540_Prot_SAP_000.pdf Phase 4 3943
found 40_NCT02513940_Prot_SAP_001.pdf Phase 4 7383
found 40_NCT02578940_Prot_000.pdf Phase 3 9558
found 40_NCT02584140_Prot_SAP_000.pdf Phase 4 10400
found 40_NCT02610140_Prot_000.pdf Phase 2 73606
found 40_NCT02659540_Prot_SAP_000.pdf Phase 1 34603
found 40_NCT02668640_Prot_000.pdf nan 9929
found 40_NCT02692040_Prot_SAP_000.pdf Phase 1 17280
found 40_NCT02697240_Prot_SAP_000.pdf Phase 1 5096
found 40_NCT02710240_Prot_SAP_000.pdf Phase 1/Phase 2 10017
found 40_NCT02744040_Prot_000.pdf Not Applicable 21146
found 40_NCT02771340_Prot_SAP_000.pdf Phase 1 36268
found 40_NCT02810340_Prot_000.pdf Phase 1 25674
found 40_NCT02815540_Prot_SAP_000.pdf Phase 1/Phase 2 2692
found 40_NCT02819440_Prot_SAP_000.pdf Phase 2 6772
found 40_NCT02858440_Prot_000.pdf Phase 3 23241
found 40_NCT02871440_Prot_SAP_000.p

In [ ]:
len(df_annotations.text.iloc[0])

In [ ]:
df_annotations.to_csv("filtered_for_phase_arms_subjects.csv.bz2")

In [ ]:
df_annotations.to_csv("/content/drive/MyDrive/data/filtered_for_phase_arms_subjects.csv.bz2")